# Imports

In [1]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
import keras
from tensorflow.keras import layers, models
import tensorflow as tf
import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

2024-05-26 05:37:13.883002: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 05:37:13.883092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 05:37:14.026497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## See the GPU avaliablity

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Load the data

In [ ]:
#Give dataset path
train_path = '/kaggle/input/adsdata/AdsData/Train/'
test_path = '/kaggle/input/adsdata/AdsData/Test/'

In [5]:
print(f"Training classes are {len(os.listdir(train_path))} and the test classes are: {len(os.listdir(test_path))}")

Training classes are 2 and the test classes are: 2


# Create our CNN Model using Vgg16 as the base model

In [8]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False 

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(512, activation='relu')
dense_layer_2 = layers.Dense(256, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
dense_layer_3 = layers.Dense(128, activation='relu')
dense_layer_4 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dropout_layer_1,
    dense_layer_3,
    dense_layer_4,
    prediction_layer
])

In [9]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

# Data Preprocessing
- Rotate
- Width Shift
- Height Shift
- Shear Range
- Zoom
- Flip

In [10]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [11]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 16000 images belonging to 2 classes.


In [12]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4000 images belonging to 2 classes.


# Adding Callbacks and Learning rate functions to the model

In [15]:
checkpoint = ModelCheckpoint(filepath='mymodel.weights.h5',verbose=2, save_best_only=True, save_weights_only = True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
callbacks = [checkpoint, reduce_lr]
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Model Training

In [16]:
model.fit(train_set, epochs=30, validation_data=test_set, callbacks=callbacks)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-26 05:44:11.428541: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.63498, expected 3.85968
2024-05-26 05:44:11.428613: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.55784, expected 5.78254
2024-05-26 05:44:11.428623: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 6.61282, expected 5.83752
2024-05-26 05:44:11.428631: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 6.34499, expected 5.56968
2024-05-26 05:44:11.428639: E external/local_xla/xla/service/gpu/buffer_comparator.cc:11

  2/500 ━━━━━━━━━━━━━━━━━━━━ 36s 73ms/step - accuracy: 0.4766 - loss: 21.7834   

I0000 00:00:1716702268.262996     184 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5174 - loss: 7.4800
Epoch 1: val_loss improved from inf to 2.10374, saving model to mymodel.weights.h5
500/500 ━━━━━━━━━━━━━━━━━━━━ 932s 2s/step - accuracy: 0.5175 - loss: 7.4747 - val_accuracy: 0.5700 - val_loss: 2.1037 - learning_rate: 1.0000e-05
Epoch 2/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5864 - loss: 2.4981
Epoch 2: val_loss improved from 2.10374 to 1.38575, saving model to mymodel.weights.h5
500/500 ━━━━━━━━━━━━━━━━━━━━ 701s 1s/step - accuracy: 0.5864 - loss: 2.4977 - val_accuracy: 0.5990 - val_loss: 1.3857 - learning_rate: 1.0000e-05
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5995 - loss: 1.7686
Epoch 3: val_loss improved from 1.38575 to 1.04052, saving model to mymodel.weights.h5
500/500 ━━━━━━━━━━━━━━━━━━━━ 696s 1s/step - accuracy: 0.5995 - loss: 1.7683 - val_accuracy: 0.6332 - val_loss: 1.0405 - learning_rate: 1.0000e-05
Epoch 4/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy

# Save our model weights

In [22]:
model.save_weights("newmodel.weights.h5")